In [1]:
!pip install tensorflow==2.2.0

     |████████████████████████████████| 516.2MB 32kB/s 
     |████████████████████████████████| 460kB 28kB/s 
     |████████████████████████████████| 3.0MB 50.0MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [11]:
import tensorflow as tf

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
#from tensorflow.keras.optimizers import Adam
from tensorflow import keras

import zipfile

In [12]:
print(tf.__version__)

2.2.0


In [13]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/vid_frame.zip", 'r')
zip_ref.extractall("/content/emotion")
zip_ref.close()

In [18]:
import os
print(len(os.listdir("/content/emotion/testing/calm")))

925


In [19]:
TRAINING_DIR = "/content/emotion/training/"
VALIDATION_DIR = "/content/emotion/validation/"

In [20]:
opt = keras.optimizers.Adam(learning_rate =0.001)
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=30)
mc = ModelCheckpoint('/content/drive/My Drive/BestResnet50.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

img_height = 224
img_width = 224
batch_size = 16
nb_epochs = 50

In [21]:
train_datagen = ImageDataGenerator(rescale=1./255)
# validation_split=0.1) # set validation split

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')
# subset='training') # set as training data

val_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = train_datagen.flow_from_directory(
    VALIDATION_DIR, # same directory as training data
    target_size= (img_height, img_width),
    batch_size= batch_size,
    class_mode='categorical')
# subset='validation') # set as validation data

Found 52314 images belonging to 8 classes.
Found 5550 images belonging to 8 classes.


In [22]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
emotions = Dense(8, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=emotions)

for layer in base_model.layers:
  layer.trainable = True

In [23]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

model.fit_generator(train_generator,
                    steps_per_epoch = train_generator.samples // batch_size,
                    validation_data = validation_generator,
                    validation_steps = validation_generator.samples // batch_size,
                    epochs = nb_epochs, callbacks=[es, mc])

model.save('/content/drive/My Drive/Resnet50')

Epoch 1/50
3269/3269 [==============================] - ETA: 0s - loss: 0.6869 - acc: 0.7570
Epoch 00001: val_acc improved from -inf to 0.36796, saving model to /content/drive/My Drive/BestResnet50.h5
3269/3269 [==============================] - 862s 264ms/step - loss: 0.6869 - acc: 0.7570 - val_loss: 2.5345 - val_acc: 0.3680
Epoch 2/50
3269/3269 [==============================] - ETA: 0s - loss: 0.3088 - acc: 0.8942
Epoch 00002: val_acc improved from 0.36796 to 0.44996, saving model to /content/drive/My Drive/BestResnet50.h5
3269/3269 [==============================] - 860s 263ms/step - loss: 0.3088 - acc: 0.8942 - val_loss: 2.0538 - val_acc: 0.4500
Epoch 3/50
3269/3269 [==============================] - ETA: 0s - loss: 0.1938 - acc: 0.9334
Epoch 00003: val_acc improved from 0.44996 to 0.50650, saving model to /content/drive/My Drive/BestResnet50.h5
3269/3269 [==============================] - 860s 263ms/step - loss: 0.1938 - acc: 0.9334 - val_loss: 1.8566 - val_acc: 0.5065
Epoch 4/50